In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
ltc_requests = requests.get("http://publicreporting.ltchomes.net/en-ca/Search_Selection.aspx")

In [3]:
ltc_requests.status_code
ltc_requests.reason

200

In [5]:
print(ltc_requests.request.headers)

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [6]:
ltc_requests.headers


{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Server': 'Microsoft-IIS/8.5', 'Set-Cookie': 'ASP.NET_SessionId=juqtevjz3b0woi12oxpsxupj; path=/; HttpOnly; SameSite=Lax', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Mon, 11 May 2020 17:57:35 GMT', 'Content-Length': '201949'}

In [8]:
ltc = BeautifulSoup(ltc_requests.text)

In [26]:
ordered_lists = ltc.find_all("ol")
ltc_list = ordered_lists[1]
home = ltc_list.find_all("a", {"class":"rsLink"})
len(home)

651

In [38]:
names = []
links = []
for each in home:
    names.append(each.get('title'))
    links.append(each.get('href'))

In [42]:
# create list with full url for each ltc home
full_links = []
for i in range(len(links)):
    full_links.append('http://publicreporting.ltchomes.net/en-ca/' + links[i])

In [43]:
# iterate through each of the links and scrape for address, city, postal code, LHIN, Administrator, Licensee, Management Firm, Home Type, Licensed Beds, Residents' Council, Family Council and Accreditation
address = []
city = []
postal_code = []
LHIN = []
for each in full_links:
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    address = address.append()
    

    

http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=C501
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2861
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=M501
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=3041
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2647
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=M536
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2692
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=0956
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2786
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2930
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2982
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2756
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=M502
http://publicreporting.ltchomes.ne

In [61]:
soup = requests.get(
"http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872")
soup = BeautifulSoup(soup.text)
address = (soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"})).getText()



'1200 Afton Drive'

In [60]:
address.getText()

'1200 Afton Drive'